In [ ]:
from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from skimage import io
from skimage.morphology import erosion, square, binary_erosion

from glob import glob

import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# image = data.hubble_deep_field()[0:500, 0:500]
image = io.imread('../Datasets/Dataset_BUSI_with_GT/benign/benign (36)_mask.png')
print(type(image))
# image = data.coins()[50:-50, 50:-50]
image_gray = rgb2gray(image) if len(image.shape) == 3 else image
# image_gray = image
print(image_gray.shape)

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=0.1)
print(blobs_log.shape)

# Compute radii in the 3rd column.
# blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
blobs_log[:, 2] = blobs_log[:, 2]

blobs_dog = blob_dog(image_gray, max_sigma=30, threshold=0.1)
print(blobs_dog.shape)
# blobs_dog[:, 2] = blobs_dog[:, 2]

blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=0.01)
print(blobs_doh.shape)
# print(blobs_doh[:,0])
# print(blobs_doh[:,1])
# print(blobs_doh[:,2])

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['red', 'red', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 5, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(image)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

nidx = idx+1
ax[nidx].set_title("OG Image")
e_image = erosion(image)
ax[nidx].imshow(image)
nidx += 1
ax[nidx].set_title("Eroded Image")
ax[nidx].imshow(e_image)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.transform import resize


image = data.coins()[50:-50, 50:-50]

# apply threshold
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))



# remove artifacts connected to image border
cleared = clear_border(bw)

# label image regions
label_image = label(cleared)
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)
# ax.imshow(bw)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 100:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle(
            (minc, minr),
            maxc - minc,
            maxr - minr,
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

In [ ]:
x = np.zeros((10,10))

In [ ]:
a, b, r = 4, 6, 3

In [ ]:
y = x

In [ ]:
y[(a-r+1):(a+r),(b-r+1):(b+r)] = 1

In [ ]:
len(y.shape)

In [ ]:
import torch

In [ ]:
yt = torch.tensor(y)
print(yt)

In [ ]:
import os
fnames = os.listdir('../Datasets/Dataset_BUSI_with_GT/benign/')
for fname in fnames:
#     print(fname)
    x = fname.split('_')
    if len(x) > 2:
        print(x)
        print(x[2].split('.')[0])

### 

###

### Description of code

Open a file. Load related mask files. Erode them individually until less than 50 and more than 20 pixels remain in the foreground. Note the co-ordinates of the eroded mask (like a bounding box). Use this as the weak mask to compute BCE Loss against. Compute BCE loss ONLY on this bounding box - training will happen only for the positive class

In [ ]:
# image = io.imread('../Datasets/Dataset_BUSI_with_GT/benign/benign (36)_mask.png')

In [ ]:
petsdir = '../Datasets/petsData/annotations/trimaps/'
busidir = '../Datasets/Dataset_BUSI_with_GT/benign/'

In [ ]:
# im_name = 'benign (195).png'
# image = io.imread('../Datasets/Dataset_BUSI_with_GT/benign/' + im_name)

In [ ]:
im_name = 'Egyptian_Mau_58.png'
image = io.imread('../Datasets/petsData/annotations/trimaps/' + im_name)

In [ ]:
def get_masks(im_name):
    
    imn = im_name.split('.')[0]
#     mask_names = glob(busidir + imn + '_mask' + '*')
    mask_names = glob(petsdir + imn + '*')
    
    return mask_names
    

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.imshow(image)

In [ ]:
# def eroded_mask(mask):
    
#     o_pixels = np.sum(mask)
#     e_mask = mask
#     pixels = np.sum(e_mask)
#     while pixels >= 50:
        
# #         e_mask = binary_erosion(e_mask, square(3))
#         e_mask = erosion(e_mask, np.ones((3,3)))
#         pixels = np.sum(e_mask)
        
#     print(pixels)
    
#     return e_mask

In [ ]:
def eroded_mask(mask):
    
    o_pixels = np.sum(mask)
#     print(o_pixels)
    print("o_pixels: ", o_pixels)
    e_mask = mask
    pixels = np.sum(e_mask)
    while pixels >= 0.01 * o_pixels:
        
#         e_mask = binary_erosion(e_mask, square(3))
        e_mask_t = erosion(e_mask, np.ones((3,3)))
        pixels = np.sum(e_mask_t)
        
        if pixels != 0:
            e_mask = e_mask_t
        
    print(pixels)
    
    return e_mask

In [ ]:
mask_names = get_masks(im_name)
print(mask_names)
print(image.shape)
# mask = np.zeros(image.shape[:2])
# mask_og = np.zeros(image.shape[:2])
mask = np.zeros((224, 224))
mask_og = np.zeros((224, 224))

masks_og = list()

for f in mask_names:
    
#     mask_f = io.imread(f)
    mask_f = io.imread(f)
    print(0 in mask_f, 1 in mask_f, 2 in mask_f, 3 in mask_f)
    print(type(mask_f))
    
    M1 = ((mask_f == 1) * 1.0)
    M2 = ((mask_f == 2) * 1.0)
    M3 = ((mask_f == 3) * 1.0)

    mask_f = M1 + M3
    
    #resizing op
#     mask_f = resize(mask_f, (224,224))
    mask_f = resize(mask_f, (222,222))
    
    mask_f = np.pad(mask_f, ((1,1),(1,1)), 'constant', constant_values=(0))
    
    print(0 in mask_f, 1 in mask_f, 2 in mask_f, 3 in mask_f)
    
#     M1 = ((mask_f == 1) * 1.0)
#     M2 = ((mask_f == 2) * 1.0)
#     M3 = ((mask_f == 3) * 1.0)

#     mask_f = M1 + M3
    
    masks_og.append(mask_f)
    ##do erosion
    
    e_mask = eroded_mask(mask_f)
    mask_og += mask_f

    mask += e_mask

In [ ]:
print(np.sum((mask_f == 3) * 1.0))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(mask)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(mask_og)
# ax.imshow(square(3))

In [ ]:
# tmask = io.imread('../Datasets/Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png')

# fig, ax = plt.subplots(figsize=(10, 10))
# ax.imshow(tmask)

In [ ]:
a = np.array([[1,2,3], [4,5,6]])
b = np.pad(a, ((1,1),(1,1)), 'constant', constant_values=(0))

In [ ]:
print(a.shape, b.shape)

In [ ]:
print(a)
print(b)